In [1]:
%pip install haystack-ai

In [ ]:
from dot_env import load_dotenv
load_dotenv()

OPENAI_DEPLOYMENT = 'gpt-4o'

# Embedding Models
You can leverage embedding models from Azure OpenAI through two components: `AzureOpenAITextEmbedder` and `AzureOpenAIDocumentEmbedder`.

To create semantic embeddings for documents, use `AzureOpenAIDocumentEmbedder` in your indexing pipeline. For generating embeddings for queries, use `AzureOpenAITextEmbedder`. Once you’ve selected the suitable component for your specific use case, initialize the component with required parameters.

Below is the example indexing pipeline with `InMemoryDocumentStore`, `AzureOpenAIDocumentEmbedder` and `DocumentWriter`:

In [3]:
from haystack import Document, Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.embedders import AzureOpenAITextEmbedder, AzureOpenAIDocumentEmbedder
from haystack.components.writers import DocumentWriter

document_store = InMemoryDocumentStore(embedding_similarity_function="cosine")

documents = [Document(content="My name is Wolfgang and I live in Berlin"),
             Document(content="I saw a black horse running"),
             Document(content="Germany has many big cities")]

indexing_pipeline = Pipeline()
indexing_pipeline.add_component("embedder", AzureOpenAIDocumentEmbedder(azure_deployment="text-embedding-ada-002"))
indexing_pipeline.add_component("writer", DocumentWriter(document_store=document_store))
indexing_pipeline.connect("embedder", "writer")

indexing_pipeline.run({"embedder": {"documents": documents}})


Embedding Texts: 100%|██████████| 1/1 [00:01<00:00,  1.77s/it]


{'embedder': {'meta': {'model': 'text-embedding-ada-002',
   'usage': {'prompt_tokens': 20, 'total_tokens': 20}}},
 'writer': {'documents_written': 3}}

# Generative Models (LLMs)
You can leverage Azure OpenAI models through two components: `AzureOpenAIGenerator` and `AzureOpenAIChatGenerator`.

To use OpenAI models deployed through Azure services for text generation, initialize a `AzureOpenAIGenerator` with `azure_deployment` and `azure_endpoint`. You can then use the `AzureOpenAIGenerator` instance in a pipeline after the `PromptBuilder`.

Below is the example of generative questions answering pipeline using RAG with `PromptBuilder` and `AzureOpenAIGenerator`:

In [ ]:
from haystack import Pipeline
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.generators import AzureOpenAIChatGenerator, AzureOpenAIGenerator

template = """
Given the following information, answer the question.

Context: 
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: What's the official language of {{ country }}?
"""
pipe = Pipeline()

pipe.add_component("retriever", InMemoryBM25Retriever(document_store=document_store))
pipe.add_component("prompt_builder", PromptBuilder(template=template))
pipe.add_component("llm", AzureOpenAIGenerator(azure_deployment=OPENAI_DEPLOYMENT))
pipe.connect("retriever", "prompt_builder.documents")
pipe.connect("prompt_builder", "llm")

pipe.run({
    "retriever": {"query": "What's the official language of {{ country }}?"},
    "prompt_builder": {
        "country": "France"
    }
})   


ImportError: cannot import name 'AzureOpenAIChatGenerator' from 'haystack.components.generators' (c:\sw\anaconda3\envs\torch\Lib\site-packages\haystack\components\generators\__init__.py)

Intento de Chat Assistant

In [ ]:

generator = AzureOpenAIChatGenerator(azure_deployment=OPENAI_DEPLOYMENT)